In [1]:
import io
import pandas as pd
from connections.aws import AWS

In [3]:
""" TEST 1: INITIALIZE CONNECTION """
# create connection
aws_connection = AWS()
assert aws_connection.connected == 0

# plug in connection
aws_connection.connect()
assert aws_connection.connected == 1

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


In [4]:
""" TEST 2: CONNECTION INFO """
assert hasattr(aws_connection, 's3')
assert aws_connection.bucket_name == 'pitch-ml'

In [5]:
""" TEST 3: S3 LISTING """
# NOTE: have to update IAM user permissions to allow S3 access
s3_objects = aws_connection.list_s3_objects()
s3_objects_pro = aws_connection.list_s3_objects(prefix='PRO/')

# confirm objects are there
assert isinstance(s3_objects, list)
assert len(s3_objects) > 0

# confirm PRO objects are there
assert isinstance(s3_objects_pro, list)
assert len(s3_objects_pro) > 0

In [ ]:
""" TEST 4: S3 LOADING """
# extract CSV objects
s3_objects_csv = [obj for obj in s3_objects if obj.endswith('.csv')]
s3_objects_trc = [obj for obj in s3_objects if obj.endswith('.trc')]

# load example from S3
test_object, test_info = aws_connection.load_s3_object(s3_objects_trc[0])

# create folder for subject
aws_connection.create_s3_folder('subjects/')        # parent subject folder
aws_connection.create_s3_folder(f'subjects/{test_info['subject_id']}/')  # subject folder

[AWS]: Created folder s3://pitch-ml/subjects/


In [7]:
test_info['subject_id']

'2609'

$\textbf{Development: Functions}$

In [8]:
from dev_trc import *
from biomech.processing.rotate import rotate_data

$\textbf{Implementation: Functions}$

- Reworked `biomech.trc` package
- Added TRC reading (header + body) to `AWS` module

In [9]:
import numpy as np

In [10]:
__markers_left__ = [
    'X19', 'Y19', 'Z19', # 19,left_bicep
    'X20', 'Y20', 'Z20', # 20,left_lateral_elbow
    'X21', 'Y21', 'Z21', # 21,left_medial_elbow
    'X22', 'Y22', 'Z22', # 22,left_forearm
    'X23', 'Y23', 'Z23', # 23,left_lateral_wrist
    'X24', 'Y24', 'Z24', # 24,left_medial_wrist
    'X25', 'Y25', 'Z25'  # 25,left_hand
]
__markers_right__ = [
    'X12', 'Y12', 'Z12', # 12,right_bicep
    'X13', 'Y13', 'Z13', # 13,right_lateral_elbow
    'X14', 'Y14', 'Z14', # 14,right_medial_elbow
    'X15', 'Y15', 'Z15', # 15,right_forearm
    'X16', 'Y16', 'Z16', # 16,right_lateral_wrist
    'X17', 'Y17', 'Z17', # 17,right_medial_wrist
    'X18', 'Y18', 'Z18'  # 18,right_hand
]

throwing_hand = 'right'
file_name = 'test.trc'
frame_rate = 480

# parse TRC body; then create metadata structure
test_body = parse_trc_body(test_object)

# create header
test_header = create_trc_header(file_name, test_body, throwing_hand, frame_rate)

# filter body to include only markers in the model
if throwing_hand == 'left':
    body = test_body[['Frame#', 'Time'] + __markers_left__]
elif throwing_hand == 'right':
    body = test_body[['Frame#', 'Time'] + __markers_right__]
else:
    raise ValueError("Invalid throwing hand specified. Use 'left' or 'right'.")

# TODO: rotate data
test_body_rotated = rotate_data(test_body, rotation_matrix=np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]]))

# write to file
# with open(file_name, 'w') as f:
#     f.write("\n".join(test_header) + "\n")
#     body.to_csv(f, sep="\t", index=True, header=False)

In [11]:
test_body_rotated

,Frame#,Time,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,...,Z43,X44,Y44,Z44,X45,Y45,Z45,X46,Y46,Z46
1,1,0.000,427.34372,1799.34082,-492.01389,358.25262,1867.26428,-568.14142,300.49475,1766.30005,...,-95.12817,338.08167,774.39655,-96.23889,393.70453,725.72693,-173.22005,347.12579,699.50525,51.60457
2,2,0.002,428.72876,1798.73535,-492.78033,358.36496,1867.13855,-567.98071,300.93030,1766.49976,...,-95.84174,337.56027,778.26910,-96.33013,393.57162,729.49426,-173.61963,346.15103,703.78522,51.72515
3,3,0.004,429.29413,1798.91638,-492.44397,358.43808,1866.95483,-567.78088,301.62436,1766.35779,...,-96.61736,337.15176,781.70789,-97.41634,393.48410,732.83105,-173.86490,345.61343,707.41772,51.18017
4,4,0.006,428.82083,1798.82629,-491.03488,358.93762,1866.73718,-567.14545,301.74283,1766.36316,...,-97.20208,336.84433,785.62689,-98.09261,393.47504,736.47821,-174.13290,344.47049,711.53796,50.36378
5,5,0.008,429.38855,1798.96436,-491.18686,359.09814,1866.70557,-566.97583,302.08365,1766.03088,...,-97.18011,336.19943,789.48090,-98.15969,393.03799,739.99323,-174.98047,343.78900,715.33752,50.05158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,598,1.244,2020.18164,1388.74060,-554.96741,1917.33081,1456.05566,-547.47815,1951.53015,1354.53784,...,-367.53247,1754.23938,35.80997,-277.82278,1676.09290,-11.45037,-328.35281,1907.67615,-22.35310,-306.48450
599,599,1.246,2022.80347,1387.41809,-555.61938,1920.25439,1455.07495,-548.49084,1954.19495,1353.58887,...,-367.56320,1754.26733,35.80242,-277.87805,1676.13123,-11.46087,-328.35733,1907.58997,-22.31590,-306.47162
600,600,1.248,2025.35205,1386.49707,-556.83173,1922.49084,1454.30408,-548.99506,1956.79163,1352.19153,...,-367.47314,1754.28821,35.75203,-277.62051,1675.86572,-11.52575,-328.16013,1907.61047,-22.28911,-306.45078
601,601,1.250,2028.72888,1384.21887,-559.50751,1925.43665,1453.53809,-549.92670,1959.59912,1351.57239,...,-367.42322,1753.99487,35.89421,-277.72546,1675.91138,-11.49981,-328.13202,1907.32251,-22.37444,-306.67355
